# BioExplorer - Vasculature
![](../bioexplorer_vasculature_banner.png)

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer, Vector3

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
status = be.reset_scene()

### Load vasculature by region

In [ ]:
assembly_name = 'Vasculature'
population_name = 'vasculature'
be.remove_assembly(assembly_name)

vasculature_assembly = be.add_assembly(assembly_name)
vasculature_model = be.add_vasculature(
    assembly_name=assembly_name,
    population_name=population_name,
    representation=be.VASCULATURE_REPRESENTATION_SEGMENT,
    color_scheme=be.VASCULATURE_COLOR_SCHEME_REGION,
    realism_level=be.VASCULATURE_REALISM_LEVEL_ALL
)

### Set materials

In [ ]:
import json
import os
from webcolors import hex_to_rgb

f = open('brain_regions.json')
obj = json.load(f)

colors=dict()
colors[0] = [0, 0, 0]
def display_child(node):
    node_id = node['id']
    color_hex_triplet = node['color_hex_triplet']
    c = hex_to_rgb('#' + color_hex_triplet)
    color = [float(c.red/256.0), float(c.green/256.0), float(c.blue/256.0)]
    colors[node_id] = color
    for child in node['children']:
        display_child(child)

for node in obj['msg']:
    display_child(node)

In [ ]:
def set_standard_colors(model_id):
    material_ids = be.get_material_ids(model_id)['ids']
    nb_material_ids = len(material_ids)
    palette = list()
    for material_id in material_ids:
        try:
            palette.append(colors[material_id])
        except KeyError as e:
            palette.append([0.5,0.5,0.5])
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=palette, specular_colors=palette)

model_ids = be.get_model_ids()['ids']
for model_id in model_ids:
    set_standard_colors(model_id)

### Rendering settings

In [ ]:
status = core.set_renderer(
    current='advanced',
    subsampling=4, max_accum_frames=64)
params = core.AdvancedRendererParams()
params.gi_ray_length = 1000.0
params.shadow_intensity = 1.0
params.soft_shadow_strength = 1.0
status = core.set_renderer_params(params)